In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np
from glob import glob


In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'C:\\Users\\NAGARJUNA\\data_sets\\tomato_crop_disease_dataset\\Train'
valid_path = 'C:\\Users\\NAGARJUNA\\data_sets\\tomato_crop_disease_dataset\\Valid'

In [10]:
MobileNetV2_model = MobileNetV2(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [11]:
# don't train existing weights
for layer in MobileNetV2_model.layers:
    layer.trainable = False

In [12]:
# useful for getting number of output classes
folders = glob('C:\\Users\\NAGARJUNA\\data_sets\\tomato_crop_disease_dataset\\Train/*')

In [13]:
# our layers - you can add more if you want
x = Flatten()(MobileNetV2_model.output)

In [14]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=MobileNetV2_model.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [16]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [17]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('C:\\Users\\NAGARJUNA\\data_sets\\tomato_crop_disease_dataset\\Train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 10593 images belonging to 10 classes.


In [19]:
test_set = test_datagen.flow_from_directory('C:\\Users\\NAGARJUNA\\data_sets\\tomato_crop_disease_dataset\\Valid',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 3133 images belonging to 10 classes.


In [20]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\NAGARJ~1\AppData\Local\Temp/ipykernel_1560/2724883676.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
663/663 [==============================] - 826s 1s/step - loss: 3.4688 - accuracy: 0.7701 - val_loss: 67.5645 - val_accuracy: 0.1379
Epoch 2/10
663/663 [==============================] - 681s 1s/step - loss: 2.1284 - accuracy: 0.8795 - val_loss: 91.6245 - val_accuracy: 0.1510
Epoch 3/10
663/663 [==============================] - 2202s 3s/step - loss: 1.5548 - accuracy: 0.9136 - val_loss: 102.0278 - val_accuracy: 0.1577
Epoch 4/10
663/663 [==============================] - 779s 1s/step - loss: 1.3009 - accuracy: 0.9286 - val_loss: 122.9366 - val_accuracy: 0.1621
Epoch 5/10
663/663 [==============================] - 799s 1s/step - loss: 1.1734 - accuracy: 0.9415 - val_loss: 141.5206 - val_accuracy: 0.1245
Epoch 6/10
663/663 [==============================] - 785s 1s/step - loss: 1.2741 - accuracy: 0.9439 - val_loss: 152.6938 - val_accuracy: 0.1612
Epoch 7/10
663/663 [==============================] - 744s 1s/step - loss: 0.9236 - accuracy: 0.9589 - val_loss: 162.5144 - val_acc